In [1]:
import pandas as pd
import polars as pl

In [2]:
#read in vessel labels data
vesselLabelsDF = pl.read_csv('fishing-vessels-v2.csv', infer_schema_length=10000)

In [3]:
vesselLabelsDF

mmsi,flag_ais,flag_registry,flag_gfw,vessel_class_inferred,vessel_class_inferred_score,vessel_class_registry,vessel_class_gfw,self_reported_fishing_vessel,length_m_inferred,length_m_registry,length_m_gfw,engine_power_kw_inferred,engine_power_kw_registry,engine_power_kw_gfw,tonnage_gt_inferred,tonnage_gt_registry,tonnage_gt_gfw,registries_listed,fishing_hours_2012,fishing_hours_2013,fishing_hours_2014,fishing_hours_2015,fishing_hours_2016,fishing_hours_2017,fishing_hours_2018,fishing_hours_2019,fishing_hours_2020
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
40120000,"""NA""","""GRC""","""GRC""","""NA""","""NA""","""trawlers""","""trawlers""","""TRUE""","""NA""","""26.3""","""26.3""","""NA""","""176.4""","""176.4""","""NA""","""133""","""133""","""EU|GFCM|IMO""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""674.1"""
444002410,"""NA""","""KOR""","""KOR""","""NA""","""NA""","""tuna_purse_seines""","""tuna_purse_seines""","""FALSE""","""NA""","""79.6""","""79.6""","""NA""","""3256.29""","""3256.29""","""NA""","""2132.33""","""2132.33""","""FFA|IMO|ISSF|KOR|WCPFC|WCPFC2""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""144.43"""
987654320,"""NA""","""PHL""","""PHL""","""NA""","""NA""","""tuna_purse_seines""","""tuna_purse_seines""","""FALSE""","""NA""","""66.4""","""66.4""","""NA""","""2219.27""","""2219.27""","""NA""","""1081""","""1081""","""FFA|ISSF|WCPFC|WCPFC2""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""545.33"""
554128120,"""NA""","""NRU""","""NRU""","""NA""","""NA""","""tuna_purse_seines""","""tuna_purse_seines""","""FALSE""","""NA""","""71.79""","""71.79""","""NA""","""2647""","""2647""","""NA""","""1350""","""1350""","""FFA|IMO|ISSF|TWN4|WCPFC""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""587.01"""
112119600,"""NA""","""USA""","""USA""","""NA""","""NA""","""fishing""","""fishing""","""TRUE""","""NA""","""24.64""","""24.64""","""NA""","""NA""","""NA""","""NA""","""85.48""","""85.48""","""IMO|USA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""318.22"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
601523000,"""ZAF""","""NA""","""ZAF""","""other_purse_seines""","""0.999""","""NA""","""other_purse_seines""","""TRUE""","""29.29""","""NA""","""29.29""","""367.53""","""NA""","""367.53""","""163.9""","""NA""","""163.9""","""NA""","""NA""","""NA""","""NA""","""225.17""","""248.62""","""276.68""","""268.53""","""138.34""","""309.7"""
601554000,"""ZAF""","""NA""","""ZAF""","""other_purse_seines""","""0.897""","""NA""","""other_purse_seines""","""TRUE""","""29.82""","""NA""","""29.82""","""362.61""","""NA""","""362.61""","""135.52""","""NA""","""135.52""","""NA""","""NA""","""26.35""","""133.38""","""89.45""","""220.58""","""168.56""","""80.66""","""186.25""","""313.66"""
601556000,"""ZAF""","""NA""","""ZAF""","""other_purse_seines""","""0.764""","""NA""","""other_purse_seines""","""TRUE""","""33.97""","""NA""","""33.97""","""625.19""","""NA""","""625.19""","""275.32""","""NA""","""275.32""","""NA""","""NA""","""61.26""","""87.62""","""154.18""","""155.5""","""217.7""","""140.74""","""203.67""","""220.57"""


In [4]:
#create primary flag field by using registry 1st and inferred 2nd
vesselLabelsDF = vesselLabelsDF.with_columns(
    pl.when(pl.col('flag_registry') != 'NA')
    .then(pl.col('flag_registry'))
    .otherwise(pl.col('flag_ais'))
    .alias('flag'))

vesselLabelsDF = vesselLabelsDF.with_columns(
    pl.when(pl.col('vessel_class_registry') != 'NA')
    .then(pl.col('vessel_class_registry'))
    .otherwise(pl.col('vessel_class_inferred'))
    .alias('class'))

In [5]:
#filter vessel labels DF for only needed fields
vesselLabelsDF = vesselLabelsDF[['mmsi', 'flag', 'class']]

In [6]:
vesselLabelsDF

mmsi,flag,class
i64,str,str
40120000,"""GRC""","""trawlers"""
444002410,"""KOR""","""tuna_purse_seines"""
987654320,"""PHL""","""tuna_purse_seines"""
554128120,"""NRU""","""tuna_purse_seines"""
112119600,"""USA""","""fishing"""
…,…,…
601523000,"""ZAF""","""other_purse_seines"""
601554000,"""ZAF""","""other_purse_seines"""
601556000,"""ZAF""","""other_purse_seines"""


In [7]:
#read in daily vessel location data
locRawDF = pl.read_csv('MMSI-daily-merged_2017_2020.csv')

In [8]:
#merge with labels DF to reference in flag and vessel class
locRawDF = locRawDF.join(vesselLabelsDF, on='mmsi', how='left')

C:\Users\Chris\AppData\Local\Temp\ipykernel_19460\3071954931.py:2: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  locRawDF = locRawDF.join(vesselLabelsDF, on='mmsi', how='left')


In [9]:
locRawDF

date,cell_ll_lat,cell_ll_lon,mmsi,hours,fishing_hours,flag,class
str,f64,f64,i64,f64,f64,str,str
"""2017-01-01""",0.0,-2.6,306097000,1.6175,0.0,"""CUW""","""tuna_purse_seines"""
"""2017-01-01""",0.0,-171.5,338539000,0.5444,0.0,"""USA""","""tuna_purse_seines"""
"""2017-01-01""",0.0,-152.6,359002000,1.3522,0.0,"""SLV""","""tuna_purse_seines"""
"""2017-01-01""",0.0,170.6,367170000,0.3972,0.0,"""USA""","""tuna_purse_seines"""
"""2017-01-01""",0.0,170.7,367170000,0.6944,0.3891,"""USA""","""tuna_purse_seines"""
…,…,…,…,…,…,…,…
"""2020-12-31""",71.7,26.1,276842000,0.2108,0.0,"""EST""","""trawlers"""
"""2020-12-31""",75.7,16.0,273350610,0.1352,0.0,"""RUS""","""trawlers"""
"""2020-12-31""",75.7,15.9,273350610,0.3288,0.0,"""RUS""","""trawlers"""


In [10]:
#filter to retain only 2019-2020
locRawDF = locRawDF.filter((pl.col('date').str.slice(0, length=4) == '2019') | (pl.col('date').str.slice(0, length=4) == '2020'))

In [11]:
locRawDF

date,cell_ll_lat,cell_ll_lon,mmsi,hours,fishing_hours,flag,class
str,f64,f64,i64,f64,f64,str,str
"""2019-01-01""",0.0,-105.6,224900000,0.2036,0.0,"""ESP""","""drifting_longlines"""
"""2019-01-01""",0.0,-105.3,224900000,0.0205,0.0,"""ESP""","""drifting_longlines"""
"""2019-01-01""",0.0,-105.5,224900000,0.9008,0.0,"""ESP""","""drifting_longlines"""
"""2019-01-01""",0.0,-105.4,224900000,0.8638,0.0,"""ESP""","""drifting_longlines"""
"""2019-01-01""",0.0,-0.3,228071900,0.7969,0.0,"""FRA""","""tuna_purse_seines"""
…,…,…,…,…,…,…,…
"""2020-12-31""",71.7,26.1,276842000,0.2108,0.0,"""EST""","""trawlers"""
"""2020-12-31""",75.7,16.0,273350610,0.1352,0.0,"""RUS""","""trawlers"""
"""2020-12-31""",75.7,15.9,273350610,0.3288,0.0,"""RUS""","""trawlers"""


In [72]:
#create unique vessel dataframe by removing all duplice mmsi's, regardless of date
uniqueVDF = locRawDF[['mmsi', 'flag', 'class']]
uniqueVDF = uniqueVDF.unique(subset=['mmsi'])

uniqueVesselCount = len(uniqueVDF)

#create flag count dataframe by counting unique flags and adding % of total field
flagCounts = uniqueVDF['flag'].value_counts()
flagCounts = flagCounts.with_columns((pl.col('count') / sum(flagCounts['count']) * 100).round(2).alias('pct'))

uniqueFlagCounts = len(flagCounts)

#aggregate all flag counts with less than 0.5% of total into an "other" category and re-run % of total
flagCounts = flagCounts.with_columns(pl.when(pl.col('pct') > 0.5)
                                     .then(pl.col('flag'))
                                     .otherwise(pl.lit('OTHER'))
                                     .alias('flag2'))

flagCounts = flagCounts[['count', 'flag2']]

flagCounts = (flagCounts.with_columns(pl.col('flag2')).group_by('flag2').sum())

flagCounts = flagCounts.with_columns((pl.col('count') / sum(flagCounts['count']) * 100).round(2).alias('pct'))
flagCounts = flagCounts.sort('pct', descending=True)

#create vessel class count dataframe by splitting and exploding vessel types, counting unique types, and adding % of total field
classCounts = (uniqueVDF.with_columns(pl.col('class').str.split('|'))
               .explode('class'))

classCounts = classCounts['class'].value_counts()
classCounts = classCounts.with_columns((pl.col('count') / sum (classCounts['count']) * 100).round(2).alias('pct'))
classCounts = classCounts.sort('pct', descending=True)

uniqueClassCounts = len(classCounts)

In [73]:
uniqueVesselCount, uniqueFlagCounts, uniqueClassCounts

(79481, 210, 18)